In [1]:
# Common Imports
from arcgis.gis import GIS
import csv

In [2]:
# Source GIS
source_gis = GIS(url="https://epndemo.maps.arcgis.com/", username="kholleran_demos")

Enter password: ········


## <font color='red'>Add Users from CSV</font>

In [3]:
# Get Roles in org
role_mgr = source_gis.users.roles
org_roles = role_mgr.all()
org_roles

[<Role name: Viewer, description: Viewer>,
 <Role name: Data Editor, description: Data Editor>,
 <Role name: Facilitator, description: Facilitator>,
 <Role name: VelocityUser, description: User with Publisher + Veloctiy Feed, RAT & BAT privileges>,
 <Role name: [author] Partner User Type, description: Author permissions for Standard Partner User Types>,
 <Role name: Author, description: Author permissions for Standard Partner User Types>,
 <Role name: DataDownload, description: Data Download test>,
 <Role name: GeoBIM_Beta, description: Geo BIM  beta test workflows>,
 <Role name: [edit] Partner User Type, description: Editor permissions for Basic Partner User Types>,
 <Role name: [view] Partner User Type, description: Default viewer permissions for Lite Partner User Types>]

In [4]:
with open("/arcgis/home/usermanagement.csv", "r") as csvfile:
    addusers = csv.reader(csvfile)
    
    # Skip the first row
    next(addusers)
        
        # Format Role
    for row in addusers:
        update_role = 'iAAAAAAAAAAAAAAA'
        for role in org_roles:
            if role.name == row[6]:
                update_role = role.role_id
                break

        # Add users
        source_gis.users.create(username=row[1][0].lower() + row[2].lower()+ "_test", password='Pleasechangeme123', firstname=row[1], lastname=row[2], email=row[0], role=update_role, user_type=row[5].lower() + 'UT')
        print("Succesfully created user {}".format(row[1][0].lower() + row[2].lower()+ "_test"))

Succesfully created user kholleran_test
Succesfully created user jrajamohan_test
Succesfully created user rfernandes_test
Succesfully created user bmccardle_test
Succesfully created user jharrington_test


## <font color='red'>Allocate Licenses</font>

#### <font color='blue'>List of Licenses</font>

In [5]:
for license in source_gis.admin.license.all():
    if "listing" in license.properties:
        print(license.properties.listing.title)

ArcGIS Survey123
ArcGIS Insights
Esri Redistricting
ArcGIS Pro
ArcGIS Drone2Map
ArcGIS Image for ArcGIS Online
ArcGIS GeoBIM
ArcGIS for Power BI
ArcGIS GeoPlanner
ArcGIS AppStudio
ArcGIS Drone2Map
ArcGIS Workforce
ArcGIS Community Analyst
Runtime Analysis 
ArcGIS Navigator
ArcGIS Collector
ArcGIS Tracker
Street Smart for ArcGIS Pro
ArcGIS Business Analyst Web and Mobile Apps
ArcGIS CityEngine


#### <font color='blue'>Assign Pro License</font>

In [6]:
pro_license = source_gis.admin.license.get('ArcGIS Pro')
pro_entitlements=['desktopAdvN']
pro_license.assign('jrajamohan_test', pro_entitlements)

True

#### <font color='blue'>Assign Pro License with Extension</font>

In [7]:
pro_entitlements_ext=['desktopAdvN', 'networkAnalystN']
pro_license.assign('kholleran_test', pro_entitlements_ext)

True

#### <font color='blue'>Assign ArcGIS Field Apps and Essential Apps Bundle</font>

In [8]:
bundles = source_gis.admin.license.bundles

In [9]:
bm = source_gis.users.get("bmccardle_test") 

for bundle in bundles:
    if bundle.properties.name in ['Field Apps', 'Essential Apps']:
        bundle.assign(bm)

## <font color='red'>Create & Assign Groups</font>

#### <font color='blue'>Create Group</font>

In [10]:
group_schema = {
    "title": "EPC 2022",
    "tags": "epc2022, group, poc",
    "description": "My very descriptive title for one client",
    "access": 'private',
    "is_invitation_only": True,
    "users_update_items": False
}

group = source_gis.groups.create_from_dict(group_schema)

#### <font color='blue'>Get Usernames</font>

In [11]:
with open("/arcgis/home/usermanagement.csv", "r") as csvfile:
    addusers = csv.reader(csvfile)
    
    # Skip the first row
    next(addusers)
    
    usernames = []
    for row in addusers:
        usernames.append(row[1][0].lower() + row[2].lower()+ "_test")

#### <font color='blue'>Invite Users</font>

In [12]:
for user in usernames:
    group.add_users(user)
    print("Succesfully added {} to group".format(user))

Succesfully added kholleran_test to group
Succesfully added jrajamohan_test to group
Succesfully added rfernandes_test to group
Succesfully added bmccardle_test to group
Succesfully added jharrington_test to group


## <font color='red'>Delete User</font>

#### <font color='blue'>Remove Entitlements</font>

In [13]:
pro_license.revoke(username='kholleran_test', entitlements=pro_entitlements_ext)
pro_license.revoke(username='jrajamohan_test', entitlements=pro_entitlements)

True

In [14]:
bundles = source_gis.admin.license.bundles
for bundle in bundles:
    if bundle.properties.name in ['Field Apps', 'Essential Apps']:
        bundle.revoke(bm)

#### <font color='blue'>Delete User</font>

In [15]:
with open("/arcgis/home/usermanagement.csv", "r") as csvfile:
    delete_users = csv.reader(csvfile)
    
    # Skip the first row
    next(delete_users)

    # Format Role
    for row in delete_users:
        deleteuser = source_gis.users.get(username=row[1][0].lower() + row[2].lower() + "_test")
        deleteuser.delete()        
        print("Succesfully deleted user {}".format(row[1][0].lower() + row[2].lower()+ "_test"))

Succesfully deleted user kholleran_test
Succesfully deleted user jrajamohan_test
Succesfully deleted user rfernandes_test
Succesfully deleted user bmccardle_test
Succesfully deleted user jharrington_test
